<a href="https://colab.research.google.com/github/saimasharleen/Active-Learning-Strategies-Across-Diverse-Machine-Learning-Models/blob/main/MNIST_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/modAL-python/modAL.git

  Cloning https://github.com/modAL-python/modAL.git to /tmp/pip-req-build-dcvh_tq_
  Running command git clone --filter=blob:none --quiet https://github.com/modAL-python/modAL.git /tmp/pip-req-build-dcvh_tq_
  Resolved https://github.com/modAL-python/modAL.git to commit bba6f6fd00dbb862b1e09259b78caf6cffa2e755
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 2.9 MB/s eta 0:00:00
  Created wheel for modAL-python: filename=modAL_python-0.4.2-py3-none-any.whl size=32650 sha256=3e000cf05fe4020aa6d7b7b235f7b73e9a3abf1ff6eff602a21ed208e5073dde
  Stored in directory: /tmp/pip-ephem-wheel-cache-vwb3vhxf/wheels/d9/fb/59/7deb61b460c1c36394cd093758986ff7d36f71352dcb2e02c5
Successfully built modAL-python


In [2]:
import numpy as np
import joblib
import collections
from sklearn.naive_bayes import MultinomialNB
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling

In [3]:
# Load MNIST data
mnist_data = MNIST('.', download=True, transform=ToTensor())
dataloader = DataLoader(mnist_data, shuffle=True, batch_size=60000)
x, y = next(iter(dataloader))


100%|██████████| 9912422/9912422 [00:00<00:00, 82201339.41it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 71805390.53it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 20374114.80it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3570858.25it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [4]:
# Preprocessing
x = x.detach().cpu().numpy().reshape(x.size(0), -1)  # Flatten the images
x = x * 255  # Scale pixel values for Naive Bayes (MultinomialNB)
y = y.detach().cpu().numpy()

In [5]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=10000, random_state=0, stratify=y)

In [6]:
# Define confusion matrix print function
def CF_Print(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    CF = confusion_matrix(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    micro_f1 = f1_score(y_test, y_pred, average='micro')
    weighted_f1 = f1_score(y_test, y_pred, average='weighted')
    print('Confusion Matrix:')
    print(CF)
    print('Accuracy: ', accuracy)
    print('Macro F1-score: ', macro_f1)
    print('Micro F1-score: ', micro_f1)
    print('Weighted F1-score: ', weighted_f1)
    print("---------------------------------")

In [7]:
# Active Learning with Naive Bayes
def AL_Retrain(n_initial, n_queries, instances):
    x_initial, x_pool, y_initial, y_pool = train_test_split(x_train, y_train, train_size=n_initial, random_state=0, stratify=y_train)

    # Initialize the learner
    learner = ActiveLearner(
        estimator=MultinomialNB(),
        query_strategy=uncertainty_sampling,
        X_training=x_initial, y_training=y_initial
    )

    print("Number of initial data: ", n_initial)
    y_pred = learner.predict(x_test)
    CF_Print(y_test, y_pred)

    # Active Learning Loop
    for idx in range(n_queries):
        print('--- Query no: ', idx+1, ' ----')
        query_idx, query_instance = learner.query(x_pool, n_instances=instances)
        learner.teach(X=x_pool[query_idx], y=y_pool[query_idx])
        print('Instance', y_pool[query_idx])
        y_pred = learner.predict(x_test)
        CF_Print(y_test, y_pred)

        # Update the pool
        x_pool = np.delete(x_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)


In [8]:
AL_Retrain(30, 10, 20)

Number of initial data:  30
Confusion Matrix:
[[685   0   0  38   0  16  43   0 204   1]
 [  0 577   4   2  19   0   2   1 512   7]
 [  2   1 646  66   2  10  78   3 172  13]
 [  3   0  31 461   0  78   6   7 404  32]
 [  0   0   4   5 399   3  46   0 240 277]
 [ 10   0   3  33  14 237  10   2 572  22]
 [  4   2   0   7   1  10 758   0 203   1]
 [ 12   3  19   6  19   1   1 598 232 153]
 [  5   1   9   4  11  65  20   0 844  16]
 [  2   1   3   3  21   3  16  11 410 522]]
Accuracy:  0.5727
Macro F1-score:  0.6050731272922876
Micro F1-score:  0.5727
Weighted F1-score:  0.6090033431889876
---------------------------------
--- Query no:  1  ----
Instance [1 3 3 3 7 4 4 7 4 3 2 4 6 6 1 3 1 9 2 4]
Confusion Matrix:
[[675   0   8  13  41  15  52   1 168  14]
 [  0 989  41  12   7   0   3   2  69   1]
 [  0  32 651 130  17   1  63  16  57  26]
 [  2   7  68 715  32  23   7  33 129   6]
 [  0   0   4   2 767   2  18   3  44 134]
 [  7   6   2  94 148 192  25   5 413  11]
 [  2  13   2   7  25 

In [9]:
AL_Retrain(30, 10, 30)

Number of initial data:  30
Confusion Matrix:
[[685   0   0  38   0  16  43   0 204   1]
 [  0 577   4   2  19   0   2   1 512   7]
 [  2   1 646  66   2  10  78   3 172  13]
 [  3   0  31 461   0  78   6   7 404  32]
 [  0   0   4   5 399   3  46   0 240 277]
 [ 10   0   3  33  14 237  10   2 572  22]
 [  4   2   0   7   1  10 758   0 203   1]
 [ 12   3  19   6  19   1   1 598 232 153]
 [  5   1   9   4  11  65  20   0 844  16]
 [  2   1   3   3  21   3  16  11 410 522]]
Accuracy:  0.5727
Macro F1-score:  0.6050731272922876
Micro F1-score:  0.5727
Weighted F1-score:  0.6090033431889876
---------------------------------
--- Query no:  1  ----
Instance [1 3 3 3 7 4 4 7 4 3 2 4 6 6 1 3 1 8 9 4 2 1 3 4 2 4 7 3 7 2]
Confusion Matrix:
[[643   0  12  35  40  10  37   0 197  13]
 [  0 984  62  16   3   0   2   1  56   0]
 [  0  21 612 148  22   1  52  12 107  18]
 [  0   9 100 745  10   3   8  21 122   4]
 [  0   1   2   6 850   0   6   2  67  40]
 [  4   8   2  97 115 147  20   3 501   6]
 [

In [10]:
AL_Retrain(50, 10, 20)

Number of initial data:  50
Confusion Matrix:
[[652   0   0   7   0  23  24   0 278   3]
 [  0 859   3   0  21   2   3   5 231   0]
 [  2   3 455  40   0  28  82   8 368   7]
 [  0   4  19 390   1  88   7   7 487  19]
 [  0   1   0   1 462   0  31   0 265 214]
 [  2   7   1  36  14 201  12   0 619  11]
 [  1  12   0   5   0   3 846   0 119   0]
 [ 12   3  20   1  56   0   0 562 211 179]
 [  1  16   1   1   2  28   6   0 914   6]
 [  0   3   0   0  82   2   6  10 436 453]]
Accuracy:  0.5794
Macro F1-score:  0.6034572609086148
Micro F1-score:  0.5794
Weighted F1-score:  0.6096149030922228
---------------------------------
--- Query no:  1  ----
Instance [6 3 9 7 2 1 4 8 1 0 1 2 5 1 7 1 7 3 4 6]
Confusion Matrix:
[[ 617    0    0    0    0    4    6    0  359    1]
 [   0 1013    1    3   14    6    3    0   83    1]
 [   0    9  430   60    1   14   75    5  396    3]
 [   6   24   19  552    0   28   12   15  343   23]
 [   0    3    0   11  428    1   22    8  309  192]
 [  10   17    

In [11]:
AL_Retrain(50, 10, 30)

Number of initial data:  50
Confusion Matrix:
[[652   0   0   7   0  23  24   0 278   3]
 [  0 859   3   0  21   2   3   5 231   0]
 [  2   3 455  40   0  28  82   8 368   7]
 [  0   4  19 390   1  88   7   7 487  19]
 [  0   1   0   1 462   0  31   0 265 214]
 [  2   7   1  36  14 201  12   0 619  11]
 [  1  12   0   5   0   3 846   0 119   0]
 [ 12   3  20   1  56   0   0 562 211 179]
 [  1  16   1   1   2  28   6   0 914   6]
 [  0   3   0   0  82   2   6  10 436 453]]
Accuracy:  0.5794
Macro F1-score:  0.6034572609086148
Micro F1-score:  0.5794
Weighted F1-score:  0.6096149030922228
---------------------------------
--- Query no:  1  ----
Instance [6 3 9 7 2 1 4 8 1 0 1 2 1 5 7 3 1 4 6 7 4 3 2 4 4 0 9 5 1 7]
Confusion Matrix:
[[ 778    0    0    0    1    5    4    0  198    1]
 [   0 1007    0    3   10    8    3    0   92    1]
 [   0    9  416   53    2   21   76    5  408    3]
 [  11   24   18  538    1   37   12    9  340   32]
 [   1    3    0    8  550    1   22    4  171  

In [12]:
AL_Retrain(100, 10, 20)

Number of initial data:  100
Confusion Matrix:
[[788   0   1   8   0  20  11   0 157   2]
 [  0 922  17  12   1   8   5   1 139  19]
 [  1   3 708  43   1  37  90   4  94  12]
 [  1   4  33 718   0  55   4   2 164  41]
 [  0   1  14   3 532   9  36   0 227 152]
 [  5   5   1  77   6 393  12   1 375  28]
 [  2   8   6   5   1  15 855   0  92   2]
 [ 14   1  19   2  40   1   0 581 183 203]
 [  0   9   1  15   4 113   3   0 786  44]
 [  3   2   4  11  40   6   7  12 281 626]]
Accuracy:  0.6909
Macro F1-score:  0.7069229302514247
Micro F1-score:  0.6909
Weighted F1-score:  0.7115762122978714
---------------------------------
--- Query no:  1  ----
Instance [4 4 9 7 7 4 4 0 3 4 5 7 3 8 4 5 9 4 1 4]
Confusion Matrix:
[[870   0   0   7   0  13   5   0  90   2]
 [  0 947  18  14   2   9   4   0 108  22]
 [  5   5 711  42   3  28  89   2  98  10]
 [  5   4  38 758   1  62   5   1 100  48]
 [  8   1  14   4 672  14  33   0 106 122]
 [  7   7   2  69  19 418  13   1 346  21]
 [  3  11   6   4   2

In [13]:
AL_Retrain(300, 10, 20)

Number of initial data:  300
Confusion Matrix:
[[ 852    0   35    2    0   23    6    0   67    2]
 [   0 1020   19    2    2    8    1    1   64    7]
 [   3    7  830   15    4    7   57    7   60    3]
 [   7   16  103  728    2   31    8   10   78   39]
 [   2    4   37    2  661   17   37    0   67  147]
 [  17    8   12  157    8  494   19    2  151   35]
 [   4   10   19    0    1   15  903    0   33    1]
 [  15   11   20    3   72    6    0  754   37  126]
 [   2   35   18   40   10   77    7    5  709   72]
 [   9    6   13   11  105    5    3   19   46  775]]
Accuracy:  0.7726
Macro F1-score:  0.7709918419241966
Micro F1-score:  0.7725999999999998
Weighted F1-score:  0.7745042553642119
---------------------------------
--- Query no:  1  ----
Instance [5 5 6 3 7 0 7 4 8 4 3 8 7 7 5 6 9 0 5 4]
Confusion Matrix:
[[ 876    0   33    1    0   14    4    0   58    1]
 [   0 1021   20    3    2    7    2    1   61    7]
 [   3    8  822   18    4    9   56    8   63    2]
 [   7  

In [14]:

AL_Retrain(300, 10, 30)

Number of initial data:  300
Confusion Matrix:
[[ 852    0   35    2    0   23    6    0   67    2]
 [   0 1020   19    2    2    8    1    1   64    7]
 [   3    7  830   15    4    7   57    7   60    3]
 [   7   16  103  728    2   31    8   10   78   39]
 [   2    4   37    2  661   17   37    0   67  147]
 [  17    8   12  157    8  494   19    2  151   35]
 [   4   10   19    0    1   15  903    0   33    1]
 [  15   11   20    3   72    6    0  754   37  126]
 [   2   35   18   40   10   77    7    5  709   72]
 [   9    6   13   11  105    5    3   19   46  775]]
Accuracy:  0.7726
Macro F1-score:  0.7709918419241966
Micro F1-score:  0.7725999999999998
Weighted F1-score:  0.7745042553642119
---------------------------------
--- Query no:  1  ----
Instance [5 5 6 3 7 0 7 4 8 3 0 9 7 1 5 5 9 4 5 4 6 5 7 7 8 5 9 7 5 8]
Confusion Matrix:
[[ 875    0   32    1    0   19    4    0   55    1]
 [   0 1030   21    3    2    7    2    1   52    6]
 [   3    9  820   17    4    8   56    7